In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import time
import gc
import sys
import pickle

print(f"Tensorflow Version: {tf.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"Numpy Version: {np.__version__}")
print(f"System Version: {sys.version}")

matplotlib.rcParams['figure.figsize'] = (17, 5)
matplotlib.rcParams['axes.grid'] = False
sns.set_style("whitegrid")


def mpl_inline(): 
    rc = plt.rcParams.copy()
    %matplotlib inline
    plt.rcParams.update(rc)
mpl_inline() #  A bug in IPython - does not pick up rcParams! (see https://github.com/jupyter/notebook/issues/3385)

In [ ]:
past_history = 240
future_target = 24
TL=past_history+future_target
# predictor index
PRED_INDEX=2 # 3rd column (RL) is what we want to predict

In [ ]:
STEP=1
def create_time_steps(length):
    return list(range(-length, 0))

   
def multi_step_plot(history, true_future, prediction):
    num_in = create_time_steps(len(history))
    num_out = len(true_future)

    plt.plot(num_in, np.array(history), label='History')
    plt.plot(np.arange(num_out)/STEP, np.array(true_future), '--gh',
           label='True Future')
    if prediction.any():
        plt.plot(np.arange(num_out)/STEP, np.array(prediction), '--rh',
                 label='Predicted Future')
    plt.legend(loc='upper left')
    plt.show()


def get_window(dataset,start,back, fwd, predictor_index):
    print(start-back, start,start,start+fwd )
    pd=dataset[start-back:start, :]
    fd=dataset[start:start+fwd, :]
    print(pd[0], pd[-1],fd[0],fd[-1])

    return pd, fd[:,predictor_index]

In [ ]:
multi_step_model=tf.keras.models.load_model('model_good.keras')
with open("model_good.param","rb") as fp:
    data_mean, data_std = pickle.load(fp)
    print(f"Mean : {data_mean} and std : {data_std}")


In [ ]:
df = pd.read_csv('data.csv', index_col=0);
dataset_= df.values
dataset = (dataset_-data_mean)/data_std

x, y=get_window(dataset,530,past_history,future_target, PRED_INDEX)
multi_step_plot(x[:,PRED_INDEX], y, np.array([0]))


In [ ]:
x, y=get_window(dataset,532,past_history,future_target, PRED_INDEX)
multi_step_plot(x[:,PRED_INDEX], y, multi_step_model.predict(np.array([x]))[0])

Now let us check some edge caeses (e.g. extremes)

In [ ]:
maxv=dataset[:,PRED_INDEX].max()
idm=np.where(dataset == maxv)[0][0]
print(idm,maxv)

In [ ]:
x, y=get_window(dataset,idm+3,past_history,future_target, PRED_INDEX)
multi_step_plot(x[:,PRED_INDEX], y, multi_step_model.predict(np.array([x]))[0])

In [ ]:
x, y=get_window(dataset,idm-137,past_history,future_target, PRED_INDEX)
multi_step_plot(x[:,PRED_INDEX], y, multi_step_model.predict(np.array([x]))[0])